https://www.kaggle.com/code/ryanholbrook/creating-features

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)




/tmp/ipykernel_33/2668033471.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


In [2]:
accidents = pd.read_csv("/kaggle/input/us-accidents/US_Accidents_March23.csv")

In [3]:
autos = pd.read_csv("/kaggle/input/automobile-dataset/Automobile_data.csv")

In [4]:
customer = pd.read_csv("/kaggle/input/ibm-watson-marketing-customer-value-data/WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv")


The concrete file cannot be read with read_csv, as it gives an encoding error.  Thus the need to install the excel reading kit (xlrd)

In [8]:
!pip install xlrd


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [12]:

concrete = pd.read_excel("/kaggle/input/concrete-comprehensive-strength/Concrete_Data.xls")

In [ ]:
autos.columns.tolist()

In [ ]:
autos[["stroke", "bore"]].head()

In [ ]:
print(autos.dtypes)

Get the indices where there is an unknown value, which cannot be used in the calculations.

In [ ]:
unknown_strokes = autos.index[autos['stroke'] == "?"].tolist()

In [ ]:
unknown_bores = autos.index[autos['bore'] == "?"].tolist()

In [ ]:
invalid_auto_data = list(set(unknown_bores + unknown_strokes))
print(invalid_auto_data)

In [ ]:
autos = autos.drop(invalid_auto_data)
autos["stroke"] = autos["stroke"].astype(float)
autos["bore"] = autos["bore"].astype(float)

In [ ]:
autos["stroke_ratio"] = autos.stroke / autos.bore

print(autos.dtypes)

In [ ]:
autos[["stroke", "bore", "stroke_ratio", "num-of-cylinders"]].head()

hah, the cylinders column is strings names of numbers...
https://stackoverflow.com/questions/493174/is-there-a-way-to-convert-number-words-to-integers

In [ ]:
!pip install word2number

In [ ]:
from word2number import w2n

print(w2n.word_to_num(autos["num-of-cylinders"][0]))

In [ ]:
autos["num-of-cylinders"] = autos.apply(lambda x: w2n.word_to_num(x["num-of-cylinders"]), axis=1)

In [ ]:
autos["num-of-cylinders"].head()

In [ ]:
# This does not work, see above...
# autos["num-of-cylinders"] = w2n.word_to_num(autos["num-of-cylinders"])

In [ ]:
autos["displacement"] = (
    np.pi * ((0.5 * autos.bore) ** 2) * autos.stroke * autos["num-of-cylinders"]
)

In [ ]:
autos["displacement"].head()

In [ ]:
print(accidents.dtypes)

In [ ]:
# If the feature has 0.0 values, use np.log1p (log(1+x)) instead of np.log
accidents["LogWindSpeed"] = accidents["Wind_Speed(mph)"].apply(np.log1p)

# Plot a comparison
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
sns.kdeplot(accidents["Wind_Speed(mph)"], fill=True, ax=axs[0])
sns.kdeplot(accidents.LogWindSpeed, fill=True, ax=axs[1]);

In [ ]:
roadway_features = ["Amenity", "Bump", "Crossing", "GiveWay",
    "Junction", "NoExit", "Railway", "Roundabout", "Station", "Stop",
    "TrafficCalming", "TrafficSignal"]
accidents["RoadwayFeatures"] = accidents[roadway_features].sum(axis=1)

accidents[roadway_features + ["RoadwayFeatures"]].head(10)

In [13]:
print(concrete.dtypes)

Cement (component 1)(kg in a m^3 mixture)                float64
Blast Furnace Slag (component 2)(kg in a m^3 mixture)    float64
Fly Ash (component 3)(kg in a m^3 mixture)               float64
Water  (component 4)(kg in a m^3 mixture)                float64
Superplasticizer (component 5)(kg in a m^3 mixture)      float64
Coarse Aggregate  (component 6)(kg in a m^3 mixture)     float64
Fine Aggregate (component 7)(kg in a m^3 mixture)        float64
Age (day)                                                  int64
Concrete compressive strength(MPa, megapascals)          float64
dtype: object


In [10]:
components = [ "Cement", "BlastFurnaceSlag", "FlyAsh", "Water",
               "Superplasticizer", "CoarseAggregate", "FineAggregate"]
concrete["Components"] = concrete[components].gt(0).sum(axis=1)

concrete[components + ["Components"]].head(10)

KeyError: "None of [Index(['Cement', 'BlastFurnaceSlag', 'FlyAsh', 'Water', 'Superplasticizer',\n       'CoarseAggregate', 'FineAggregate'],\n      dtype='object')] are in the [columns]"